# Introdução

Neste projeto, trabalharemos com dados do [CIA World Factbook](https://www.cia.gov/the-world-factbook/), um compêndio de estatísticas sobre todos os países da Terra. O Factbook contém informações demográficas como as seguintes:

- population - a população global.
- population_growth - a taxa anual de crescimento populacional, como uma porcentagem.
- area - a área total de terra e água.

Vamos utilizar SQL para analisar dados desse database.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

# Overview dos dados

Vamos analisaras colunas e as cinco primeiras linhas da tabela.

In [2]:
%%sql
SELECT *
  FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Aqui estão as descrições de algumas das colunas:

- name - o nome do país.
- area - A área total de terra e mar do país.
- population - a população do país.
- population_growth - O crescimento da população do país como uma porcentagem.
- birth_rate - A taxa de natalidade do país, ou o número de nascimentos por ano por 1.000 pessoas.
- death_rate - A taxa de mortalidade do país, ou o número de mortes por ano por 1.000 pessoas.
- area- A área total do país (terra e água).
- area_land - A área terrestre do país em quilômetros quadrados.
- area_water - A área hídrica do país em quilômetros quadrados.

# Estatísticas


In [3]:
%%sql
SELECT min(population), max(population),
        min(population_growth), max(population_growth)
        FROM facts;

Done.


min(population),max(population),min(population_growth),max(population_growth)
0,7256490011,0.0,4.02


Vemos algumas coisas interessantes nas estatísticas acima:

- Existe um país com uma população de 0
- Existe um país com uma população de 7256490011 (ou mais de 7,2 bilhões de pessoas)

Vamos usar subconsultas para ampliar apenas esses países, sem usar os valores específicos.

In [4]:
%%sql
SELECT * FROM facts WHERE population == (
        SELECT MIN(population) FROM facts
);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [5]:
%%sql
SELECT * FROM facts WHERE population == (
        SELECT MAX(population) FROM facts
);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Parece que a tabela contém uma linha para todo o mundo, o que explica a população de mais de 7,2 bilhões. Também parece que a tabela contém uma linha para a Antártica, o que explica a população de 0.

# Explorando a média da população e da área

Temos um país com quase 1.4 bilhões de pessoas:

In [6]:
%%sql
SELECT MIN(population), MAX(population),
       MIN(population_growth), MAX(population_growth) 
  FROM facts
 WHERE name <> 'World';

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


Vemos que a população média gira em torno de 32 milhões e a área média é de 555 mil quilômetros quadrados.

In [7]:
%%sql
SELECT AVG(population),
       AVG(area) 
  FROM facts
WHERE name <> 'World';

Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


# Encontrando países densamente povoados

Agora vamos identificar os países que possuem o seguinte:

- Valores acima da média para a população.
- Valores abaixo da média para a área.

In [8]:
%%sql
SELECT * FROM facts
    WHERE population > (SELECT AVG(population)
                           FROM facts WHERE name <> 'World')
    AND area < (SELECT AVG(area)
                   FROM facts WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


# Qual país tem mais gente? 

In [11]:
%%sql
SELECT * FROM facts 
WHERE population == (SELECT MAX(population) FROM facts
                    WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


# Qual país tem a maior taxa de crescimento?

In [13]:
%%sql
SELECT * FROM facts 
WHERE population_growth == (SELECT MAX(population_growth) FROM facts
                    WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


# Quais países têm as maiores proporções de água por terra? Quais países têm mais água do que terra?

In [19]:
%%sql
SELECT name, 
       CAST(area_water AS FLOAT)/area_land AS water_land_ratio
  FROM facts
 WHERE water_land_ratio > 1;

Done.


name,water_land_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832


In [16]:
%%sql
SELECT * FROM facts 
WHERE area_water > area_land;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


# Quais países adicionarão mais pessoas às suas populações no próximo ano?

In [22]:
%%sql
SELECT name, population *(population_growth/100) AS pop FROM facts
WHERE name != 'World'
ORDER BY pop DESC
LIMIT 3;

Done.


name,pop
India,15270686.124799998
China,6153684.246
Nigeria,4448270.372


# Quais países têm uma taxa de mortalidade mais alta do que a taxa de natalidade?

In [27]:
%%sql
SELECT name,death_rate,birth_rate, (death_rate-birth_rate) 
as diff FROM facts WHERE death_rate > birth_rate
ORDER BY diff DESC;

Done.


name,death_rate,birth_rate,diff
Bulgaria,14.44,8.92,5.52
Serbia,13.66,9.08,4.58
Latvia,14.31,10.0,4.3100000000000005
Lithuania,14.27,10.1,4.17
Ukraine,14.46,10.72,3.74
Hungary,12.73,9.16,3.5700000000000003
Germany,11.42,8.47,2.9499999999999993
Slovenia,11.37,8.42,2.9499999999999993
Romania,11.9,9.14,2.76
Croatia,12.18,9.45,2.7300000000000004


# Quais países têm a maior proporção de população / área e como ela se compara à lista que encontramos na tela anterior?

Podemos perceber que Monaco aparece nas duas estatísticas.

In [30]:
%%sql
SELECT name, population/area AS prop FROM facts
ORDER BY prop DESC
LIMIT 5;

Done.


name,prop
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
